# PREPARE CSV FILE

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel('../metadata/NOH_Data_surgery.xlsx')
df

,Patient #,FNA biopsy,Surgery diagnosis,Surgery diagnosis in number
0,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1
1,2,Benign (Bethesda II),Benign,0
2,3,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1
3,4,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1
4,5,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1
...,...,...,...,...
127,128,Atypia of undetermine (Bethesda III),Benign thyroid follicular neoplasm,0
128,129,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1
129,130,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1
130,131,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1


In [3]:
df['Surgery diagnosis in number'].value_counts(normalize=True)

Surgery diagnosis in number
1    0.772727
0    0.227273
Name: proportion, dtype: float64

In [4]:

df['Bethesda'] = df['FNA biopsy '].apply(lambda x: re.findall(r'Bethesda (\w+)',x)[0])
df

,Patient #,FNA biopsy,Surgery diagnosis,Surgery diagnosis in number,Bethesda
0,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V
1,2,Benign (Bethesda II),Benign,0,II
2,3,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V
3,4,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V
4,5,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V
...,...,...,...,...,...
127,128,Atypia of undetermine (Bethesda III),Benign thyroid follicular neoplasm,0,III
128,129,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V
129,130,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V
130,131,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1,VI


In [5]:
df['Bethesda'].value_counts()

Bethesda
V      52
VI     48
II     12
III     8
IV      6
I       6
Name: count, dtype: int64

In [6]:
latin = {
    'I': 1,
    'II': 2,
    'III': 3,
    'IV': 4,
    'V': 5,
    'VI': 6
}

In [7]:
df['Bethesda_num'] = df['Bethesda'].apply(lambda x:latin[x])
df['label'] = df['Surgery diagnosis in number']
df

,Patient #,FNA biopsy,Surgery diagnosis,Surgery diagnosis in number,Bethesda,Bethesda_num,label
0,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1
1,2,Benign (Bethesda II),Benign,0,II,2,0
2,3,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1
3,4,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1
4,5,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1
...,...,...,...,...,...,...,...
127,128,Atypia of undetermine (Bethesda III),Benign thyroid follicular neoplasm,0,III,3,0
128,129,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1
129,130,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1
130,131,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1,VI,6,1


In [8]:
df.to_csv('../metadata/NOH_surgery_groundtruth.csv')

Prepare Train/Val/Test dataset

In [9]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df['label'], test_size=0.3)

In [10]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
skf = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
skf.get_n_splits(df['label'],df['label'])
for i, (train_index, test_index) in enumerate(skf.split(df['label'],df['label'])):
    df.loc[train_index,f'fold_{i}'] = 'train'
    df.loc[test_index,f'fold_{i}'] = 'test'

In [11]:
df

,Patient #,FNA biopsy,Surgery diagnosis,Surgery diagnosis in number,Bethesda,Bethesda_num,label,fold_0,fold_1,fold_2,fold_3,fold_4
0,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,test,test,test,test,train
1,2,Benign (Bethesda II),Benign,0,II,2,0,test,train,train,train,train
2,3,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,train,train,train,train,test
3,4,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,train,train,train,train,train
4,5,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,train,train,train,test,train
...,...,...,...,...,...,...,...,...,...,...,...,...
127,128,Atypia of undetermine (Bethesda III),Benign thyroid follicular neoplasm,0,III,3,0,train,train,train,train,train
128,129,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,train,train,train,train,train
129,130,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,train,train,train,test,train
130,131,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1,VI,6,1,train,train,train,train,test


In [12]:
df.to_csv('../metadata/NOH_surgery_patient_dataset.csv')

Read image files

In [13]:
import os
import pathlib

In [14]:
p = pathlib.Path('../data/NOH')
all_folders = list(p.iterdir())
all_folders = [i for i in all_folders if i.is_dir()]
all_folders[:3]

[PosixPath('../data/NOH/042. NTH.R'),
 PosixPath('../data/NOH/098. DTG'),
 PosixPath('../data/NOH/096. BTHH.R')]

In [15]:
rows = []
for f in all_folders:
    patient_id = int(re.findall(r'(\d+).',f.name)[0])
    all_imgs = list(f.iterdir())
    all_imgs = [i for i in all_imgs if i.name.startswith('IMG') and i.name.endswith('.jpg')]
    for img in all_imgs:
        rows.append((patient_id, str(img)))

In [16]:
image_dataset = pd.DataFrame(rows, columns=['patient_id', 'image_path'])
image_dataset

,patient_id,image_path
0,42,../data/NOH/042. NTH.R/IMG_20230119_092341.jpg
1,42,../data/NOH/042. NTH.R/IMG_20230119_092349.jpg
2,42,../data/NOH/042. NTH.R/IMG_20230119_092307.jpg
3,42,../data/NOH/042. NTH.R/IMG_20230119_091227.jpg
4,42,../data/NOH/042. NTH.R/IMG_20230119_092238.jpg
...,...,...
2043,41,../data/NOH/041. NTH.L/IMG_20230119_090927.jpg
2044,41,../data/NOH/041. NTH.L/IMG_20230119_091023.jpg
2045,41,../data/NOH/041. NTH.L/IMG_20230119_090844.jpg
2046,41,../data/NOH/041. NTH.L/IMG_20230119_090904.jpg


In [17]:
image_dataset.to_csv('../metadata/NOH_surgery_image_path.csv')

# PREPARE CSV FILE

In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('../metadata/NOH_surgery_patient_dataset.csv',index_col=0)
df1 = pd.read_csv('../metadata/NOH_surgery_image_path.csv',index_col=0)

In [20]:
df = df.merge(df1, how='inner', left_on='Patient # ', right_on='patient_id')
df

,Patient #,FNA biopsy,Surgery diagnosis,Surgery diagnosis in number,Bethesda,Bethesda_num,label,fold_0,fold_1,fold_2,fold_3,fold_4,patient_id,image_path
0,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,test,test,test,test,train,1,../data/NOH/001.NguyeThiLan- Right/IMG_2022121...
1,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,test,test,test,test,train,1,../data/NOH/001.NguyeThiLan- Right/IMG_2022121...
2,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,test,test,test,test,train,1,../data/NOH/001.NguyeThiLan- Right/IMG_2022121...
3,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,test,test,test,test,train,1,../data/NOH/001.NguyeThiLan- Right/IMG_2022121...
4,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,test,test,test,test,train,1,../data/NOH/001.NguyeThiLan- Right/IMG_2022121...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,132,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1,VI,6,1,train,train,test,train,train,132,../data/NOH/132. LTN/IMG_20230228_134752.jpg
2044,132,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1,VI,6,1,train,train,test,train,train,132,../data/NOH/132. LTN/IMG_20230228_134749.jpg
2045,132,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1,VI,6,1,train,train,test,train,train,132,../data/NOH/132. LTN/IMG_20230228_134822.jpg
2046,132,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1,VI,6,1,train,train,test,train,train,132,../data/NOH/132. LTN/IMG_20230228_134601.jpg


In [21]:
df[['patient_id', 'label']].drop_duplicates()['label'].value_counts().sort_index()

label
0     30
1    102
Name: count, dtype: int64

In [22]:
def new_label(x):
    return x

df['new_label'] = df['label'].apply(lambda x: new_label(x))
df

,Patient #,FNA biopsy,Surgery diagnosis,Surgery diagnosis in number,Bethesda,Bethesda_num,label,fold_0,fold_1,fold_2,fold_3,fold_4,patient_id,image_path,new_label
0,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,test,test,test,test,train,1,../data/NOH/001.NguyeThiLan- Right/IMG_2022121...,1
1,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,test,test,test,test,train,1,../data/NOH/001.NguyeThiLan- Right/IMG_2022121...,1
2,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,test,test,test,test,train,1,../data/NOH/001.NguyeThiLan- Right/IMG_2022121...,1
3,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,test,test,test,test,train,1,../data/NOH/001.NguyeThiLan- Right/IMG_2022121...,1
4,1,Thyroid papillary carcinoma (Bethesda V),Thyroid papillary carcinoma,1,V,5,1,test,test,test,test,train,1,../data/NOH/001.NguyeThiLan- Right/IMG_2022121...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,132,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1,VI,6,1,train,train,test,train,train,132,../data/NOH/132. LTN/IMG_20230228_134752.jpg,1
2044,132,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1,VI,6,1,train,train,test,train,train,132,../data/NOH/132. LTN/IMG_20230228_134749.jpg,1
2045,132,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1,VI,6,1,train,train,test,train,train,132,../data/NOH/132. LTN/IMG_20230228_134822.jpg,1
2046,132,Thyroid papillary carcinoma (Bethesda VI),Thyroid papillary carcinoma,1,VI,6,1,train,train,test,train,train,132,../data/NOH/132. LTN/IMG_20230228_134601.jpg,1


In [23]:
df[['patient_id', 'new_label']].drop_duplicates()['new_label'].value_counts().sort_index()

new_label
0     30
1    102
Name: count, dtype: int64

In [ ]:
df.to_csv('../metadata/NOH_surgery_2c.csv')